In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from tqdm import tqdm
from scipy import stats

In [2]:
device = torch.device('cuda:0')

In [3]:
# data_transforms = {
#     'train': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.CenterCrop(224),
# #         transforms.RandomResizedCrop(224),
# #         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.Resize(256),
# #         transforms.CenterCrop(224),
# #         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [4]:
class ImageDataset(Dataset):
    def __init__(self, paths, labels, transform=None, stride = 1, max_size=None):
        self.paths = paths[::stride]
        if max_size is not None:
            self.paths = self.paths[:max_size]
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = Image.open(img_name).convert('RGB')
        
        img_tensor = self.transform(img)

        return img_tensor, self.labels[idx]

In [5]:
train_file = 'commercials/data/train.txt'
val_file = 'commercials/data/val.txt'
test_file = 'commercials/data/test.txt'

In [6]:
def read_file(filename):
    paths = []
    labels = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            video, image, label = line.split(' ')
            paths.append('commercials/images/{}/{:04d}.jpg'.format(video, int(image)))
            labels.append(int(label))
            
    return paths, labels

In [7]:
train_paths, Y_train = read_file(train_file)
val_paths, Y_val = read_file(val_file)
test_paths, Y_test = read_file(test_file)

In [8]:
image_datasets = {
    'train': ImageDataset(train_paths, Y_train, transform = data_transforms['train'],
                          stride=1, max_size=None),
    'val_train': ImageDataset(val_paths, Y_val, transform = data_transforms['train'],
                       stride=1, max_size=None),
    'val': ImageDataset(val_paths, Y_val, transform = data_transforms['val'],
                       stride=1, max_size=None),
    'test': ImageDataset(test_paths, Y_test, transform = data_transforms['val'],
                        stride=1, max_size=None)
}

In [9]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=4,
                                              shuffle='train' in x,
                                              num_workers=4,
                                              pin_memory=True)
                                              for x in image_datasets}

In [10]:
dataset_sizes = {
    x: len(image_datasets[x])
    for x in image_datasets
}

In [11]:
dataset_sizes

{'train': 64130, 'val_train': 9479, 'val': 9479, 'test': 7496}

In [12]:
def safe_divide(a, b):
    return a / b if b > 0 else 0

In [13]:
def train_model(model, criterion, optimizer, scheduler, train_dl, val_dl, test_dl=None,
                num_epochs=25, return_best=False, verbose=True, log_file=None):
    print(train_dl, val_dl, test_dl)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    best_test_acc = 0.0
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0
    
    phases = ['train', 'val', 'test'] if test_dl is not None else ['train', 'val']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
                dl = dataloaders[train_dl]
                dataset_size = dataset_sizes[train_dl]
                
            elif phase == 'val':
                model.eval()   # Set model to evaluate mode
                dl = dataloaders[val_dl]
                dataset_size = dataset_sizes[val_dl]
            else:
                model.eval()
                dl = dataloaders[test_dl]
                dataset_size = dataset_sizes[test_dl]

            running_loss = 0.0
            running_corrects = 0
            true_positives = 0.
            true_negatives = 0.
            false_positives = 0.
            false_negatives = 0.

            # Iterate over data.
            for inputs, labels in dl:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.where(
                        outputs >= 0.,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                    target = torch.where(
                        labels >= 0.5,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device)
                    )
                    loss = criterion(outputs.view(target.shape), target)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                label_vals = torch.where(
                    labels >= 0.5,
                    torch.tensor([1.]).to(device),
                    torch.tensor([0.]).to(device)
                )
                correct = preds.view(label_vals.shape) == label_vals.data
                running_corrects += torch.sum(correct)
                
                true_positives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                true_negatives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_positives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_negatives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
            
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
            epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
            epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

            if verbose:
                print('{} Loss: {:.4f} Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc, epoch_pre, epoch_recall, epoch_f1))
                print('TP: {} TN: {} FP: {} FN: {}'.format(
                    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))
            if log_file is not None:
                log_file.write('Phase: {0}\t'
                               'Epoch: [{1}/{2}]\t'
                               'Loss: {loss_c:.4f}\t'
                               'Acc: {acc:.4f}\t'
                               'Pre: {pre:.4f}\t'
                               'Rec: {rec:.4f}\t'
                               'F1: {f1:.4f}\t'
                               'TP: {tp} '
                               'TN: {tn} '
                               'FP: {fp} '
                               'FN: {fn}\n'.format(
                                   phase, epoch + 1, num_epochs, loss_c=epoch_loss,
                                   acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                                   f1=epoch_f1, tp=int(true_positives.data), tn=int(true_negatives.data),
                                   fp=int(false_positives.data), fn=int(false_negatives.data)
                               ))
                log_file.flush()

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_acc = epoch_acc
                best_f1 = epoch_f1
                best_precision = epoch_pre
                best_recall = epoch_recall
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test' and best_epoch == epoch:
                best_test_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    if return_best:
        print('Best epoch: {}'.format(best_epoch))
        print('Best val Acc: {:4f}'.format(best_acc))
        print('Best val Pre: {:4f}'.format(best_precision))
        print('Best val Rec: {:4f}'.format(best_recall))
        print('Best val F1: {:4f}'.format(best_f1))
        print('Test Acc: {:4f}'.format(best_test_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
    return model

In [ ]:
path = 'models/transfer_learning'
for seed in range(5):
    torch.manual_seed(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)

    model_ts = model_ts.to(device)

#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(device))
#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.]).to(device))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'val_train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=False)
        torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

val_train val test
Epoch 0/24
----------
train Loss: 0.3549 Acc: 0.8533 Pre: 0.7553 Rec: 0.7331 F1: 0.7441
TP: 2022.0 TN: 6066.0 FP: 655.0 FN: 736.0
val Loss: 0.1232 Acc: 0.9625 Pre: 0.9203 Rec: 0.9540 F1: 0.9368
TP: 2631.0 TN: 6493.0 FP: 228.0 FN: 127.0
test Loss: 0.2267 Acc: 0.9238 Pre: 0.8414 Rec: 0.9355 F1: 0.8860
TP: 2218.0 TN: 4707.0 FP: 418.0 FN: 153.0

Epoch 1/24
----------
train Loss: 0.2512 Acc: 0.9056 Pre: 0.8369 Rec: 0.8390 F1: 0.8380
TP: 2314.0 TN: 6270.0 FP: 451.0 FN: 444.0
val Loss: 0.1567 Acc: 0.9456 Pre: 0.8559 Rec: 0.9775 F1: 0.9127
TP: 2696.0 TN: 6267.0 FP: 454.0 FN: 62.0
test Loss: 0.3208 Acc: 0.8877 Pre: 0.7547 Rec: 0.9553 F1: 0.8433
TP: 2265.0 TN: 4389.0 FP: 736.0 FN: 106.0

Epoch 2/24
----------
test Loss: 0.2969 Acc: 0.8901 Pre: 0.7687 Rec: 0.9334 F1: 0.8430
TP: 2213.0 TN: 4459.0 FP: 666.0 FN: 158.0

Epoch 3/24
----------
train Loss: 0.2029 Acc: 0.9213 Pre: 0.8598 Rec: 0.8716 F1: 0.8657
TP: 2404.0 TN: 6329.0 FP: 392.0 FN: 354.0
val Loss: 0.0796 Acc: 0.9750 Pre: 

test Loss: 0.1975 Acc: 0.9400 Pre: 0.8912 Rec: 0.9228 F1: 0.9068
TP: 2188.0 TN: 4858.0 FP: 267.0 FN: 183.0

Training complete in 84m 5s
val_train val test
Epoch 0/24
----------
train Loss: 0.3494 Acc: 0.8579 Pre: 0.7580 Rec: 0.7516 F1: 0.7548
TP: 2073.0 TN: 6059.0 FP: 662.0 FN: 685.0
val Loss: 0.1126 Acc: 0.9693 Pre: 0.9533 Rec: 0.9405 F1: 0.9469
TP: 2594.0 TN: 6594.0 FP: 127.0 FN: 164.0
test Loss: 0.2164 Acc: 0.9318 Pre: 0.8796 Rec: 0.9089 F1: 0.8940
TP: 2155.0 TN: 4830.0 FP: 295.0 FN: 216.0

Epoch 1/24
----------
train Loss: 0.2504 Acc: 0.9077 Pre: 0.8395 Rec: 0.8441 F1: 0.8418
TP: 2328.0 TN: 6276.0 FP: 445.0 FN: 430.0
val Loss: 0.1087 Acc: 0.9707 Pre: 0.9658 Rec: 0.9322 F1: 0.9487
TP: 2571.0 TN: 6630.0 FP: 91.0 FN: 187.0
test Loss: 0.1750 Acc: 0.9458 Pre: 0.9190 Rec: 0.9089 F1: 0.9139
TP: 2155.0 TN: 4935.0 FP: 190.0 FN: 216.0

Epoch 2/24
----------
train Loss: 0.2197 Acc: 0.9158 Pre: 0.8523 Rec: 0.8597 F1: 0.8560
TP: 2371.0 TN: 6310.0 FP: 411.0 FN: 387.0
val Loss: 0.1395 Acc: 0.9659

test Loss: 0.2070 Acc: 0.9386 Pre: 0.8699 Rec: 0.9477 F1: 0.9071
TP: 2247.0 TN: 4789.0 FP: 336.0 FN: 124.0

Epoch 24/24
----------
train Loss: 0.1263 Acc: 0.9525 Pre: 0.9101 Rec: 0.9286 F1: 0.9192
TP: 2561.0 TN: 6468.0 FP: 253.0 FN: 197.0
val Loss: 0.0453 Acc: 0.9855 Pre: 0.9859 Rec: 0.9641 F1: 0.9749
TP: 2659.0 TN: 6683.0 FP: 38.0 FN: 99.0
test Loss: 0.1870 Acc: 0.9506 Pre: 0.9216 Rec: 0.9224 F1: 0.9220
TP: 2187.0 TN: 4939.0 FP: 186.0 FN: 184.0

Training complete in 84m 20s
val_train val test
Epoch 0/24
----------
train Loss: 0.3447 Acc: 0.8602 Pre: 0.7671 Rec: 0.7462 F1: 0.7565
TP: 2058.0 TN: 6096.0 FP: 625.0 FN: 700.0
val Loss: 0.1445 Acc: 0.9570 Pre: 0.9454 Rec: 0.9043 F1: 0.9244
TP: 2494.0 TN: 6577.0 FP: 144.0 FN: 264.0
test Loss: 0.2167 Acc: 0.9301 Pre: 0.9020 Rec: 0.8739 F1: 0.8877
TP: 2072.0 TN: 4900.0 FP: 225.0 FN: 299.0

Epoch 1/24
----------
train Loss: 0.2632 Acc: 0.8977 Pre: 0.8204 Rec: 0.8299 F1: 0.8252
TP: 2289.0 TN: 6220.0 FP: 501.0 FN: 469.0
val Loss: 0.0981 Acc: 0.972

test Loss: 0.1928 Acc: 0.9424 Pre: 0.8824 Rec: 0.9435 F1: 0.9119
TP: 2237.0 TN: 4827.0 FP: 298.0 FN: 134.0

Epoch 23/24
----------
train Loss: 0.1253 Acc: 0.9537 Pre: 0.9181 Rec: 0.9231 F1: 0.9206
TP: 2546.0 TN: 6494.0 FP: 227.0 FN: 212.0
val Loss: 0.0464 Acc: 0.9869 Pre: 0.9758 Rec: 0.9793 F1: 0.9776
TP: 2701.0 TN: 6654.0 FP: 67.0 FN: 57.0
test Loss: 0.1863 Acc: 0.9462 Pre: 0.8974 Rec: 0.9372 F1: 0.9169
TP: 2222.0 TN: 4871.0 FP: 254.0 FN: 149.0

Epoch 24/24
----------
train Loss: 0.1282 Acc: 0.9504 Pre: 0.9124 Rec: 0.9177 F1: 0.9150
TP: 2531.0 TN: 6478.0 FP: 243.0 FN: 227.0
val Loss: 0.0487 Acc: 0.9855 Pre: 0.9723 Rec: 0.9782 F1: 0.9752
TP: 2698.0 TN: 6644.0 FP: 77.0 FN: 60.0
test Loss: 0.1877 Acc: 0.9452 Pre: 0.8911 Rec: 0.9418 F1: 0.9157
TP: 2233.0 TN: 4852.0 FP: 273.0 FN: 138.0

Training complete in 84m 30s
val_train val test
Epoch 0/24
----------
train Loss: 0.3524 Acc: 0.8558 Pre: 0.7577 Rec: 0.7415 F1: 0.7495
TP: 2045.0 TN: 6067.0 FP: 654.0 FN: 713.0
val Loss: 0.1118 Acc: 0.9701

test Loss: 0.1910 Acc: 0.9446 Pre: 0.9025 Rec: 0.9249 F1: 0.9136
TP: 2193.0 TN: 4888.0 FP: 237.0 FN: 178.0

Epoch 22/24
----------
train Loss: 0.1201 Acc: 0.9565 Pre: 0.9171 Rec: 0.9351 F1: 0.9260
TP: 2579.0 TN: 6488.0 FP: 233.0 FN: 179.0
val Loss: 0.0475 Acc: 0.9866 Pre: 0.9803 Rec: 0.9735 F1: 0.9769
TP: 2685.0 TN: 6667.0 FP: 54.0 FN: 73.0
test Loss: 0.2015 Acc: 0.9414 Pre: 0.8930 Rec: 0.9258 F1: 0.9091
TP: 2195.0 TN: 4862.0 FP: 263.0 FN: 176.0

Epoch 23/24
----------
train Loss: 0.1221 Acc: 0.9570 Pre: 0.9221 Rec: 0.9307 F1: 0.9264
TP: 2567.0 TN: 6504.0 FP: 217.0 FN: 191.0
val Loss: 0.0477 Acc: 0.9866 Pre: 0.9810 Rec: 0.9728 F1: 0.9769
TP: 2683.0 TN: 6669.0 FP: 52.0 FN: 75.0
test Loss: 0.1929 Acc: 0.9440 Pre: 0.9000 Rec: 0.9258 F1: 0.9127
TP: 2195.0 TN: 4881.0 FP: 244.0 FN: 176.0

Epoch 24/24
----------
train Loss: 0.1235 Acc: 0.9564 Pre: 0.9256 Rec: 0.9246 F1: 0.9251
TP: 2550.0 TN: 6516.0 FP: 205.0 FN: 208.0
val Loss: 0.0541 Acc: 0.9825 Pre: 0.9619 Rec: 0.9786 F1: 0.9702
TP: 2699.0 

# Smooth Results

In [17]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [18]:
path = 'models/transfer_learning'

log_file_img = open(os.path.join(
    path, 'test_results_image_classifier.log'), 'w')
log_file_smoothed = open(os.path.join(
    path, 'test_results_smoothed.log'), 'w')

for seed in range(5):
    print(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts.load_state_dict(torch.load(
        os.path.join(path, 'seed_{}.pth'.format(seed))
    ))
    model_ts.to(device)
    criterion = nn.BCEWithLogitsLoss()
    
    model = model_ts.eval()   # Set model to evaluate mode
    dl = dataloaders['test']
    dataset_size = dataset_sizes['test']
    
    running_corrects = 0
    true_positives = 0.
    true_negatives = 0.
    false_positives = 0.
    false_negatives = 0.

    i = 0

    predictions = []
    gt_labels = []
    # Iterate over data.
    for inputs, labels in dl:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            preds = torch.where(
                outputs >= 0.,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
            target = torch.where(
                labels >= 0.5,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device)
            )

        predictions += preds.cpu().numpy().tolist()
        gt_labels += labels.cpu().numpy().tolist()

        # statistics
        label_vals = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        correct = preds.view(label_vals.shape) == label_vals.data
        running_corrects += torch.sum(correct)

        true_positives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        true_negatives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_positives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_negatives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )

        num_fp = torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        )

    #     if num_fp > 0:
    #         print(num_fp)
    #         print(torch.where(
    #             (correct == 0.) * (label_vals == 0.),
    #             torch.tensor([1.]).to(device),
    #             torch.tensor([0.]).to(device)))
    #         print(i)
    #         out = torchvision.utils.make_grid(inputs)
    # #         imshow(out, title=preds.cpu().numpy().tolist())
    #         imshow(out)

        i += 1

    epoch_acc = running_corrects.double() / dataset_size
    epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
    epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
    epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

    print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
        epoch_acc, epoch_pre, epoch_recall, epoch_f1))
    print('TP: {} TN: {} FP: {} FN: {}'.format(
        true_positives.data, true_negatives.data, false_positives.data, 
        false_negatives.data))
    
    log_file_img.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                       f1=epoch_f1, tp=int(true_positives.data),
                       tn=int(true_negatives.data),
                       fp=int(false_positives.data), fn=int(false_negatives.data)
                   ))
    log_file_img.flush()

    predictions = [p[0] for p in predictions]

    smoothed_preds = smooth_predictions(predictions, 3)

    print("Smoothed stats:")
    print(acc_prf1(smoothed_preds, gt_labels))
    
    sm_acc, sm_pre, sm_rec, sm_f1, sm_tp, sm_tn, sm_fp, sm_fn = acc_prf1(
        smoothed_preds, gt_labels)
    
    log_file_smoothed.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=sm_acc, pre=sm_pre, rec=sm_rec,
                       f1=sm_f1, tp=sm_tp,
                       tn=sm_tn,
                       fp=sm_fp, fn=sm_fn
                   ))
    log_file_smoothed.flush()

0
Acc: 0.9400 Pre: 0.8912 Rec: 0.9228 F1: 0.9068
TP: 2188.0 TN: 4858.0 FP: 267.0 FN: 183.0
Smoothed stats:
(0.9340981856990395, 0.8813490451036164, 0.9148038802193167, 0.8977649006622517, 2169, 4833, 292, 202)
1
Acc: 0.9506 Pre: 0.9216 Rec: 0.9224 F1: 0.9220
TP: 2187.0 TN: 4939.0 FP: 186.0 FN: 184.0
Smoothed stats:
(0.9450373532550693, 0.9145154464663563, 0.9114297764656263, 0.912970004224757, 2161, 4923, 202, 210)
2
Acc: 0.9452 Pre: 0.8911 Rec: 0.9418 F1: 0.9157
TP: 2233.0 TN: 4852.0 FP: 273.0 FN: 138.0
Smoothed stats:
(0.9389007470651014, 0.8883475436459602, 0.9228173766343315, 0.9052544476623914, 2188, 4850, 275, 183)
3
Acc: 0.9344 Pre: 0.8609 Rec: 0.9452 F1: 0.9011
TP: 2241.0 TN: 4763.0 FP: 362.0 FN: 130.0
Smoothed stats:
(0.9288954108858057, 0.8601097178683386, 0.9257697174188106, 0.8917326833231769, 2195, 4768, 357, 176)
4
Acc: 0.9338 Pre: 0.8692 Rec: 0.9308 F1: 0.8990
TP: 2207.0 TN: 4793.0 FP: 332.0 FN: 164.0
Smoothed stats:
(0.9303628601921025, 0.8678869876641464, 0.91986503584

# Understand Mistakes

In [29]:
path = 'models/transfer_learning'
model_ts = models.resnet50(pretrained=True)
num_ftrs = model_ts.fc.in_features
model_ts.fc = nn.Linear(num_ftrs, 1)
model_ts.load_state_dict(torch.load(
    os.path.join(path, 'seed_0.pth')
))
model_ts.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.]).to(device))

In [30]:
model = model_ts.eval()   # Set model to evaluate mode
dl = dataloaders['test']
dataset_size = dataset_sizes['test']

In [31]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(str(title))
    plt.pause(0.001)  # pause a bit so that plots are updated

In [32]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [33]:
running_corrects = 0
true_positives = 0.
true_negatives = 0.
false_positives = 0.
false_negatives = 0.

i = 0

predictions = []
gt_labels = []
# Iterate over data.
for inputs, labels in dl:
    inputs = inputs.to(device)
    labels = labels.to(device).float()

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        preds = torch.where(
            outputs >= 0.,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        target = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        
    predictions += preds.cpu().numpy().tolist()
    gt_labels += labels.cpu().numpy().tolist()

    # statistics
    label_vals = torch.where(
        labels >= 0.5,
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device)
    )
    correct = preds.view(label_vals.shape) == label_vals.data
    running_corrects += torch.sum(correct)

    true_positives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    true_negatives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_positives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_negatives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    
    num_fp = torch.sum(
    torch.where(
        (correct == 0.) * (label_vals == 0.),
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device))
    )

#     if num_fp > 0:
#         print(num_fp)
#         print(torch.where(
#             (correct == 0.) * (label_vals == 0.),
#             torch.tensor([1.]).to(device),
#             torch.tensor([0.]).to(device)))
#         print(i)
#         out = torchvision.utils.make_grid(inputs)
# #         imshow(out, title=preds.cpu().numpy().tolist())
#         imshow(out)
    
    i += 1

epoch_acc = running_corrects.double() / dataset_size
epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
    epoch_acc, epoch_pre, epoch_recall, epoch_f1))
print('TP: {} TN: {} FP: {} FN: {}'.format(
    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))

predictions = [p[0] for p in predictions]

smoothed_preds = smooth_predictions(predictions, 3)

print("Smoothed stats:")
print(acc_prf1(smoothed_preds, gt_labels))

Acc: 0.9421 Pre: 0.8904 Rec: 0.9317 F1: 0.9106
TP: 2209.0 TN: 4853.0 FP: 272.0 FN: 162.0
Smoothed stats:
(0.9327641408751334, 0.8783948115119579, 0.9139603542808942, 0.8958247209590741, 2167, 4825, 300, 204)
